In [1]:
'''
We ingest the data.ods and reformat to
client_id \t path \t sentence_id \t sentence \t locale

Here
client_id is a unique identifier for the song
path is a path to the song
sentence_id is a unique identifier for the sentence
sentence is the Gaelic text
sentence_domain=up_votes=down_votes=age=gender=accents=variant
locale is set to ga-IE

The client_id can be extracted from the filename "song_x,..wav"
The sentence_id can be extracted from the "..., phrase_xx..wav"

The filenames can be coupled to the filenames via the phrase_number, which is in on the data.ods
'''


'\nWe ingest the data.ods and reformat to\nclient_id \t path \t sentence_id \t sentence \t locale\n\nHere\nclient_id is a unique identifier for the song\npath is a path to the song\nsentence_id is a unique identifier for the sentence\nsentence is the Gaelic text\nsentence_domain=up_votes=down_votes=age=gender=accents=variant\nlocale is set to ga-IE\n\nThe client_id can be extracted from the filename "song_x,..wav"\nThe sentence_id can be extracted from the "..., phrase_xx..wav"\n\nThe filenames can be coupled to the filenames via the phrase_number, which is in on the data.ods\n'

In [2]:
%load_ext autoreload
%autoreload 2
import os
import re
import hashlib
import pandas as pd

In [4]:
os.chdir('/media/bramiozo/DATA-FAST/TTS/tts_models/gle/tts-vits-cv-ga_seanos/datasets')

In [5]:
lyrics_original =  pd.read_excel('lyrics.ods')

In [6]:
lyrics_original.dtypes

phrase_number     int64
sentence         object
Title            object
English          object
dtype: object

In [ ]:
hashlib.md5(b"bladiebla").hexdigest()

In [7]:
filenames = os.listdir('clips')

file_df = pd.DataFrame([{'client_id': hashlib.md5(b""+(f"{(s.split(',')[1]).strip()}").encode("latin1")).hexdigest(),
  'path': s,
  'phrase_number': int(s.split(',')[1].strip().split("_")[1])
  } for s in filenames])

In [8]:
final_df = lyrics_original[['phrase_number', 'sentence']].merge(file_df, how='inner', on='phrase_number')

In [9]:
final_df = final_df.assign(locale='ga-IE')
final_df = final_df.assign(sentence_domain='')
final_df = final_df.assign(up_votes='')
final_df = final_df.assign(down_votes='')
final_df = final_df.assign(age='')
final_df = final_df.assign(gender='')
final_df = final_df.assign(accents='')
final_df = final_df.assign(variant='')

In [10]:
final_df

,phrase_number,sentence,client_id,path,locale,sentence_domain,up_votes,down_votes,age,gender,accents,variant
0,1,"'Sé do bheatha, a bhean ba léanmhar,",129cfd61c6fc3bb81772f877f970a04a,"song_1,phrase_1, ime_range0-5.wav",ga-IE,,,,,,,
1,2,"Do b' é ár gcreach thú bheith i ngéibheann,",e0e6411a24c6e8c1677fc2a878117074,"song_1,phrase_2,time_5-10.wav",ga-IE,,,,,,,
2,3,"Do dhúiche bhreá i seilbh méirleach,",fbded93e097bfa4d66d42beeeb86c53a,"song_1,phrase_3, time_10-14.wav",ga-IE,,,,,,,
3,4,Is tú díolta leis na Gallaibh.,c4f77dadd822c530b9bf6d78d61470fc,"song_1, phrase_4,time_14-18.wav",ga-IE,,,,,,,
4,5,"Óró, sé do bheatha 'bhaile,",7e7850af2cd4aaad8a087f04def21b92,"song_1, phrase_5,time_18-22.wav",ga-IE,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
87,88,Go bhfaca na daoine é ach níl fhios cén t-am.,472ac0d2decc09913234520c418ea9e6,"song_5, phrase_88,time_41-45.wav",ga-IE,,,,,,,
88,89,"Ó, níorbh é sin ab ait liom ar theacht ráithe ...",0f24a22a23380f04381d1608c0759629,"song_5, phrase_89,time_45-48.wav",ga-IE,,,,,,,
89,90,"A bheith ag déanamh íospairt istigh ar mo shráid,",eb73183fef1ef1d93f8bae0a3a38416e,"song_5, phrase_90,time_48-52.wav",ga-IE,,,,,,,
90,91,"Go mbíonn sé ina chat is ina mhada san oíche,",dd0fe1f490b91f416511a414b37c5e1a,"song_5, phrase_91,time_52-55.wav",ga-IE,,,,,,,


In [11]:
final_df = final_df.assign(sentence_id = final_df.sentence.apply(lambda s: hashlib.md5(s.encode("utf-8")).hexdigest() ))

In [12]:
final_df

,phrase_number,sentence,client_id,path,locale,sentence_domain,up_votes,down_votes,age,gender,accents,variant,sentence_id
0,1,"'Sé do bheatha, a bhean ba léanmhar,",129cfd61c6fc3bb81772f877f970a04a,"song_1,phrase_1, ime_range0-5.wav",ga-IE,,,,,,,,6970c4d40220f150755cc0a753f3859d
1,2,"Do b' é ár gcreach thú bheith i ngéibheann,",e0e6411a24c6e8c1677fc2a878117074,"song_1,phrase_2,time_5-10.wav",ga-IE,,,,,,,,40509e3be0e46f6a95e8ed6f447b9754
2,3,"Do dhúiche bhreá i seilbh méirleach,",fbded93e097bfa4d66d42beeeb86c53a,"song_1,phrase_3, time_10-14.wav",ga-IE,,,,,,,,a4a80f2f5bed810c2eed208e701f8e8c
3,4,Is tú díolta leis na Gallaibh.,c4f77dadd822c530b9bf6d78d61470fc,"song_1, phrase_4,time_14-18.wav",ga-IE,,,,,,,,d1fc86e629698e54af56eceede1eb52b
4,5,"Óró, sé do bheatha 'bhaile,",7e7850af2cd4aaad8a087f04def21b92,"song_1, phrase_5,time_18-22.wav",ga-IE,,,,,,,,daa08a4b1ae3e4b39a437155f43bca99
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,88,Go bhfaca na daoine é ach níl fhios cén t-am.,472ac0d2decc09913234520c418ea9e6,"song_5, phrase_88,time_41-45.wav",ga-IE,,,,,,,,3f06c62e4dcd2588c3e74fda02dd5284
88,89,"Ó, níorbh é sin ab ait liom ar theacht ráithe ...",0f24a22a23380f04381d1608c0759629,"song_5, phrase_89,time_45-48.wav",ga-IE,,,,,,,,b03a6aeaecb0c1874136e1f3ec575e72
89,90,"A bheith ag déanamh íospairt istigh ar mo shráid,",eb73183fef1ef1d93f8bae0a3a38416e,"song_5, phrase_90,time_48-52.wav",ga-IE,,,,,,,,2f191e5cad946b452d065219f015ba49
90,91,"Go mbíonn sé ina chat is ina mhada san oíche,",dd0fe1f490b91f416511a414b37c5e1a,"song_5, phrase_91,time_52-55.wav",ga-IE,,,,,,,,c0ffecd9589dd8fd10f5f9a7a89992e7


In [13]:
columns = ['client_id', 'path', 'sentence', 'sentence_id', 'sentence_domain', 'up_votes', 'down_votes', 'age', 'gender', 'accents', 'variant', 'locale']
final_df[columns].to_csv('train.tsv', sep='\t', index=False)

In [ ]:
import wave

def get_wav_duration(filepath):
    with wave.open(filepath, 'rb') as wav_file:
        frames = wav_file.getnframes()
        rate = wav_file.getframerate()
        duration = frames / float(rate)
        return int(duration * 1000)  #

In [ ]:
duration_df = []
for f in filenames:
    duration = get_wav_duration(os.path.join('clips', f))
    duration_df.append({'clip': f,  'duration[ms': duration})
    
pd.DataFrame(duration_df).to_csv('clip_durations.tsv', sep='\t', index=False)